In [278]:
from collections import defaultdict
from statistics import mean
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from util import get_preprocessed_train_data

In [279]:
data = get_preprocessed_train_data()
data.describe()

,Unnamed: 0,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G_avg
count,355.000000,355.000000,355.000000,355.000000,355.000000,355.000000,355.000000,333.000000,355.000000,355.000000,355.000000,355.000000,355.000000,355.000000,355.000000,355.000000,355.00000,355.000000
mean,177.000000,16.698592,2.740845,2.540845,1.430986,2.028169,0.338028,3.939940,3.219718,3.138028,1.484507,2.332394,3.557746,5.667606,1.933627,1.918486,1.90088,1.917664
std,102.623909,1.274200,1.092033,1.079022,0.695139,0.829741,0.746814,0.903138,1.009646,1.127765,0.880846,1.294030,1.403754,8.126955,0.203979,0.233361,0.28347,0.228374
min,0.000000,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,1.562500,1.250000,1.25000,1.354167
25%,88.500000,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,1.750000,1.812500,1.81250,1.770833
50%,177.000000,17.000000,3.000000,3.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,1.937500,1.937500,1.93750,1.916667
75%,265.500000,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,2.062500,2.062500,2.06250,2.083333
max,354.000000,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,2.437500,2.437500,2.50000,2.458333


In [280]:
numeric_features = ['age', 'traveltime', 'absences']
categorical_features = ['school', 'address', 'famsize', 'Medu', 'Fedu', 'studytime', 'failures', 'schoolsup', 'famsup', 'higher', 'internet', 'freetime', 'goout', 'Dalc', 'Walc', 'health' ]
categorical_features = ['studytime']

In [281]:
def train_and_predict(numeric_features, categorical_features, verbose=0):

    df = data[['G3'] + numeric_features + categorical_features].dropna()

    X_data = df.drop(columns=['G3'])[
        numeric_features + categorical_features
    ]
    y_data = df['G3'].to_numpy()

    pipeline = Pipeline([
        ('pre', make_column_transformer((OneHotEncoder(handle_unknown='ignore'), categorical_features), remainder='passthrough')),
        ('clf', LinearRegression())
    ])

    pipeline.fit(X_data, y_data)

    y_pred_train = pipeline.predict(X_data)
    y_pred_cv = cross_val_predict(pipeline, X_data, y_data, cv=5)

    return y_data, y_pred_cv, y_pred_train


def display_results(y_data, y_dev_pred):
    for name, f in {'mae': mean_absolute_error,
                    'mse': mean_squared_error}.items():
        print(name + '\t', f(y_data, y_dev_pred))



In [282]:
y_data, y_pred_cv, y_pred_train = train_and_predict(
    ['age', 'traveltime', 'absences'],
    []
)

print("On Train Set")
display_results(y_data, y_pred_train)
print("On Validation Set (Cross Validation)")
display_results(y_data, y_pred_cv)

# Same as prev notebook

On Train Set
mae	 0.2080961850297854
mse	 0.07722089167412391
On Validation Set (Cross Validation)
mae	 0.21099255200696587
mse	 0.07966561203969015


In [283]:
y_data, y_pred_cv, y_pred_train = train_and_predict(
    ['age', 'traveltime', 'absences'],
    [ 'school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'guardian', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel'],
)

print("On Train Set")
display_results(y_data, y_pred_train)
print("On Validation Set (Cross Validation)")
display_results(y_data, y_pred_cv)

# Why is this getting worse?

On Train Set
mae	 0.18367396255201776
mse	 0.05713150893101234
On Validation Set (Cross Validation)
mae	 0.22067593478126302
mse	 0.08319697123122163


In [284]:
y_data, y_pred_cv, y_pred_train = train_and_predict(
    ['age', 'absences'],
    ['studytime', 'failures']
)

print("On Train Set")
display_results(y_data, y_pred_train)
print("On Validation Set (Cross Validation)")
display_results(y_data, y_pred_cv)

On Train Set
mae	 0.19621281873602164
mse	 0.06700763075366947
On Validation Set (Cross Validation)
mae	 0.20351003307002266
mse	 0.07265859860712855
